In [1]:
import os
from random import shuffle
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
path = 'Dataset\Manga109'

In [3]:
with open('{}\{}'.format(path, 'books.txt')) as file:
    book_list = file.readlines()
book_list = [bookname.strip() for bookname in book_list]
shuffle(book_list)

In [4]:
def convert_coords(w, h, xmin, ymin, xmax, ymax):
    xmin, xmax = int(xmin), int(xmax)
    ymin, ymax = int(ymin), int(ymax)
    
    absolute_w = (xmax-xmin)
    absolute_h = (ymax-ymin)
    xmid = absolute_w/2 + int(xmin)
    ymid = absolute_h/2 + int(ymin)
    return "0 {} {} {} {}".format(xmid/w, ymid/h, absolute_w/w, absolute_h/h)

In [6]:
# For each book
for count, bookname in enumerate(tqdm(book_list)):
    # Process annotiation
    annotation_path = '{}\\annotations\{}.xml'.format(path, bookname)
    with open(annotation_path, encoding='utf8') as a_file:
        content = ''.join(a_file.readlines())
        
    soup = BeautifulSoup(content, 'xml')
    
    # For each page in book
    for page in soup.find_all('page'):
        records = []
        # Get classes from page
        texts = page.findChildren('text')

        # Check if class exist in page, if not then, continue to the next page
        if not texts:
            continue

        filepath = '{}\images\{}\{}.jpg'.format(path, bookname, page.get('index').zfill(3))
        
        width, height = int(page.get('width')), int(page.get('height'))
        
        for text in texts:
            (xmin,ymin,xmax,ymax) = text.get('xmin'), text.get('ymin'), text.get('xmax'), text.get('ymax')
            record = convert_coords(width, height, xmin, ymin, xmax, ymax)
            records.append(record)
        
        # Copy filepath
        os.system('copy {} yolo_annot\{}-{}.jpg'.format(filepath, bookname, page.get('index').zfill(3)))
        
        # Write to yolo annotation file
        with open('yolo_annot/{}-{}.txt'.format(bookname, page.get('index').zfill(3)), 'w') as f:
            for record in records:
                f.write(record + "\n")
                
        # process 5 pages per book
        if int(page.get('index')) >= 5:
            break

100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [00:17<00:00,  6.13it/s]
